In [1]:
# imports
from pathlib import Path
import os

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u

import ccdproc as ccdp
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# find out what image type the biases are called
bias_images = ccdp.ImageFileCollection('data/2025.0224')
bias_images.summary['file', 'object']

file,object
str22,str10
SHD_20250224.0001.fits,skyflat
SHD_20250224.0002.fits,skyflat
SHD_20250224.0003.fits,skyflat
SHD_20250224.0004.fits,j0916-26
SHD_20250224.0005.fits,j0916-26
SHD_20250224.0006.fits,mgab253
SHD_20250224.0007.fits,sgr1231-42
SHD_20250224.0008.fits,bias


In [16]:
from astropy.io import fits

hdul = fits.open('data/2025.0224/SHD_20250224.0008.fits')
hdul.info()

Filename: data/2025.0224/SHD_20250224.0008.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     118   (256, 256, 30)   int16 (rescales to uint16)   


In [115]:
#create master bias

# path to all bias images
bias_path = Path('bias_images')
bias_images = ccdp.ImageFileCollection(bias_path)
calibrated_biases = bias_images.files_filtered(imagetyp='Bias Frame', include_path=True)

# combine bias images to create master bias
combined_bias = ccdp.combine(calibrated_biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             unit='adu'
                            )

# give masterbias header of combined
combined_bias.meta['combined'] = True

# name masterbias
combined_bias.write('masterbias.fits')

In [183]:
# Rename all .fit files to .fits in the input directory (attempting to avoid formatting error)
for file_name in os.listdir('raw_images'):
    if file_name.endswith('.fit'):
        old_path = os.path.join('raw_images', file_name)
        new_path = os.path.join('raw_images', file_name.replace(".fit", ".fits"))
        os.rename(old_path, new_path)
        print(f"Renamed: {old_path} -> {new_path}")

Renamed: raw_images/FlatField 4x4 B 1.000secs 00025477.fit -> raw_images/FlatField 4x4 B 1.000secs 00025477.fits
Renamed: raw_images/FlatField 4x4 B 1.000secs 00025470.fit -> raw_images/FlatField 4x4 B 1.000secs 00025470.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025485.fit -> raw_images/FlatField 4x4 V 1.000secs 00025485.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025484.fit -> raw_images/FlatField 4x4 V 1.000secs 00025484.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025486.fit -> raw_images/FlatField 4x4 V 1.000secs 00025486.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025487.fit -> raw_images/FlatField 4x4 V 1.000secs 00025487.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025483.fit -> raw_images/FlatField 4x4 V 1.000secs 00025483.fits
Renamed: raw_images/FlatField 4x4 V 1.000secs 00025482.fit -> raw_images/FlatField 4x4 V 1.000secs 00025482.fits
Renamed: raw_images/FlatField 4x4 B 1.000secs 00025481.fit -> raw_images/FlatField 4x4 B 1.000se

In [15]:
# subtract master bias from all other images

# load masterbias
masterbias_path = 'masterbias.fits'
masterbias = CCDData.read(masterbias_path, unit='adu', format='fits')

# directories for raw and bias subtracted images
input_directory = 'raw_flats/raw_science'
output_directory = 'biassub_images'
os.makedirs(output_directory, exist_ok=True)

# subtract masterbias each FITS file
for file_name in os.listdir(input_directory):
    file_path = os.path.join(input_directory, file_name)

    # process only .fits files
    if not file_name.endswith(".fit"):
        print(f"Skipping non-FITS file: {file_name}")
        continue
    
    # read the FITS file
    image = CCDData.read(file_path, unit='adu', format='fits')

    # subtract masterbias
    biassub_image = ccdp.subtract_bias(image, masterbias)
    
    # rename output file and put into folder
    biassub_mean = np.mean(biassub_image.data)
    print(file_name, biassub_mean)
    new_file_name = f"{'biassub_'}{file_name}"
    output_path = os.path.join(output_directory, new_file_name)

    # save the corrected image
    biassub_image.write(output_path, overwrite=True)
    print(f"Processed and saved: {output_path}")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Light 4x4 8.000secs V NGC 2516 00025549.fit 461.55670359965404
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs V NGC 2516 00025549.fit
Light 4x4 8.000secs B NGC 2516 00025555.fit 291.09686368296565
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs B NGC 2516 00025555.fit
Light 4x4 8.000secs B NGC 2516 00025554.fit 291.77687082737964
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs B NGC 2516 00025554.fit
Light 4x4 8.000secs V NGC 2516 00025548.fit 479.42301708201103
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs V NGC 2516 00025548.fit
Light 4x4 8.000secs B NGC 2516 00025553.fit 292.99784028803697
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs B NGC 2516 00025553.fit
Light 4x4 8.000secs B NGC 2516 00025552.fit 298.4951411526321
Processed and saved: biassub_images/biassub_Light 4x4 8.000secs B

In [9]:
import os
os.getcwd()

'/Users/kendall/Desktop/astro_capetown/A61/Lab 3 Data'

In [169]:
# find out what image type the darks are called

dark_images = ccdp.ImageFileCollection('biassub_darks')
dark_images.summary['file', 'imagetyp']

file,imagetyp
str40,str10
biassub_Dark 4x4 8.000secs 00025562.fits,Dark Frame
biassub_Dark 4x4 8.000secs 00025563.fits,Dark Frame
biassub_Dark 4x4 8.000secs 00025564.fits,Dark Frame


In [175]:
#create master dark

# path to biassub dark images
dark_path = Path('biassub_darks')
dark_images = ccdp.ImageFileCollection(dark_path)
calibrated_darks = dark_images.files_filtered(imagetyp='Dark Frame', include_path=True)

# combine dark images to create master dark
combined_dark = ccdp.combine(calibrated_darks,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std,
                             unit='adu'
                            )

# give masterdark header of combined
combined_dark.meta['combined'] = True

# name masterdark
combined_dark.write('masterdark.fits')

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


In [191]:
# find out what image type the flats are called

flat_fields = ccdp.ImageFileCollection('biassub_flats')
flat_fields.summary['file', 'imagetyp']

file,imagetyp
str47,str10
biassub_FlatField 4x4 B 1.000secs 00025470.fits,Flat Field
biassub_FlatField 4x4 B 1.000secs 00025477.fits,Flat Field
biassub_FlatField 4x4 B 1.000secs 00025478.fits,Flat Field
biassub_FlatField 4x4 B 1.000secs 00025479.fits,Flat Field
biassub_FlatField 4x4 B 1.000secs 00025480.fits,Flat Field
biassub_FlatField 4x4 B 1.000secs 00025481.fits,Flat Field
biassub_FlatField 4x4 V 1.000secs 00025482.fits,Flat Field
biassub_FlatField 4x4 V 1.000secs 00025483.fits,Flat Field
biassub_FlatField 4x4 V 1.000secs 00025484.fits,Flat Field


In [51]:
# create masterflat for B filter

# directory path for B filter images
B_input_directory = 'biassub_flats/biassub_B'

# initialize a list to hold CCDData objects
scaled_B_images = []

# scale fits images by median value
for file_name in os.listdir(B_input_directory):
    file_path = os.path.join(B_input_directory, file_name)

    # read fits file
    B_image = CCDData.read(file_path, unit="adu", format="fits")

    # calculate the median pixel value
    median_B_value = np.median(B_image.data)

    # scale image by dividing by median 
    scaled_B_image = B_image.copy()
    scaled_B_image.data = scaled_B_image.data / median_B_value

    # append the scaled image to the list
    scaled_B_images.append(scaled_B_image)

# combine the scaled images by taking the median
master_Bflat = ccdp.combine(scaled_B_images, 
                            method='median', 
                            sigma_clip=False,
                           )

# calculate the mean value of the combined master flat
mean_B_value = np.mean(master_Bflat.data)

# normalize the master flat by dividing by its mean value
normalized_master_Bflat_data = master_Bflat.data / mean_B_value
print("Mean of master_Bflat:", np.mean(normalized_master_Bflat_data.data))
print("Min value in master_Bflat:", np.min(normalized_master_Bflat.data))
print("Number of zero pixels:", np.sum(normalized_master_Bflat.data == 0))
print("Number of very small pixels (<1e-3):", np.sum(normalized_master_Bflat.data < 1e-3))


# convert master B flat to CCDData object
normalized_master_Bflat = CCDData(data=normalized_master_Bflat_data, unit=master_Bflat.unit, meta=master_Bflat.meta)

# name normalized master B flat
normalized_master_Bflat.write('master_Bflat2.fits')

print(f"Master B flat field image saved")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Mean of master_Bflat: 1.0000000000000002
Min value in master_Bflat: -0.004655346786017435
Number of zero pixels: 1053
Number of very small pixels (<1e-3): 19516
Master B flat field image saved


In [43]:
# create masterflat for V filter

# directory path for V filter images
V_input_directory = 'biassub_flats/biassub_V'

# initialize a list to hold CCDData objects
scaled_V_images = []

# scale fits images by median value
for file_name in os.listdir(V_input_directory):
    file_path = os.path.join(V_input_directory, file_name)

    # read fits file
    V_image = CCDData.read(file_path, unit="adu", format="fits")

    # calculate the median pixel value
    median_V_value = np.median(V_image.data)

    # scale image by dividing by median 
    scaled_V_image = V_image.copy()
    scaled_V_image.data = scaled_V_image.data / median_V_value

    # append the scaled image to the list
    scaled_V_images.append(scaled_V_image)

# combine the scaled images by taking the median
master_Vflat = ccdp.combine(scaled_V_images, 
                            method='median', 
                            sigma_clip=False,
                           )

# calculate the mean value of the combined master flat
mean_V_value = np.mean(master_Vflat.data)

# normalize the master flat by dividing by its mean value
normalized_master_Vflat_data = master_Vflat.data / mean_V_value
print("Mean of master_Vflat:", np.mean(normalized_master_Vflat_data.data))

# convert master V flat to CCDData object
normalized_master_Vflat = CCDData(data=normalized_master_Vflat_data, unit=master_Vflat.unit, meta=master_Vflat.meta)

# name normalized master V flat
normalized_master_Vflat.write('master_Vflat1.fits')

print(f"Master V flat field image saved")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
Mean of master_Vflat: 0.9999999999999999
Master V flat field image saved


In [57]:
# reduce raw data by subtracting dark and flat fielding

# paths for masterdark and master flat fields
masterdark_path = 'masterdark.fits'
masterdark = CCDData.read(masterdark_path, unit='adu', format='fits')

master_Bflat_path = 'master_Bflat.fits'
master_Bflat = CCDData.read(master_Bflat_path, unit='adu', format='fits')
master_Vflat_path = 'master_Vflat.fits'
master_Vflat = CCDData.read(master_Vflat_path, unit='adu', format='fits')

# set minimum threshold for flats
safe_Bflat = master_Bflat.copy()
safe_Bflat.data[safe_Bflat.data < 1e-3] = 1  # Avoid division by tiny values
safe_Vflat = master_Vflat.copy()
safe_Vflat.data[safe_Vflat.data < 1e-3] = 1  # Avoid division by tiny values

# directories
input_directory = 'biassub_images'
output_directory = 'cleaned_images_1'
os.makedirs(output_directory, exist_ok=True)

# specify exposure time
exposure_time = 8 * u.s

# reduce raw images
for file_name in os.listdir(input_directory):
    file_path = os.path.join(input_directory, file_name)

    #use master B flat to reduce B images
    if 'B' in file_name:
        B_raw = CCDData.read(file_path, unit='adu', format='fits')
        B_dark_subtracted = ccdp.subtract_dark(B_raw, masterdark,
                                               dark_exposure=exposure_time, 
                                               data_exposure=exposure_time,
                                               exposure_unit=u.second
                                               )
        B_dark_mean = np.mean(B_dark_subtracted.data)
        print('dark sub', file_name, B_dark_mean)
        B_flat_fielded = ccdp.flat_correct(B_dark_subtracted,
                                              safe_Bflat, 
                                              norm_value=1
                                             )
        B_flat_mean = np.mean(B_flat_fielded.data)
        print('flat fielded', file_name, B_flat_mean)
        
        # rename output file and put into folder
        new_file_name = f"{'cleaned_'}{file_name}"
        output_path = os.path.join(output_directory, new_file_name)
        
        # save the cleaned image
        B_flat_fielded.write(output_path, overwrite=True)
        print(f"Processed and saved: {output_path}")

    #use master V flat to reduce V images
    if 'V' in file_name:
        V_raw = CCDData.read(file_path, unit='adu', format='fits')
        V_dark_subtracted = ccdp.subtract_dark(V_raw, masterdark,
                                               dark_exposure=exposure_time, 
                                               data_exposure=exposure_time,
                                               exposure_unit=u.second
                                               )
        V_dark_mean = np.mean(V_dark_subtracted.data)
        print('dark sub', file_name, V_dark_mean)
        V_flat_fielded = ccdp.flat_correct(V_dark_subtracted,
                                           safe_Vflat, 
                                           norm_value=1
                                          )
        V_flat_mean = np.mean(V_flat_fielded.data)
        print('flat fielded', file_name, V_flat_mean)
        
        # rename output file and put into folder
        new_file_name = f"{'cleaned_'}{file_name}"
        output_path = os.path.join(output_directory, new_file_name)

        # save the cleaned image
        V_flat_fielded.write(output_path, overwrite=True)
        print(f"Processed and saved: {output_path}")

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
dark sub biassub_Light 4x4 8.000secs V NGC 2516 00025548.fit 473.2051686000985
flat fielded biassub_Light 4x4 8.000secs V NGC 2516 00025548.fit 467.58513625326685
Processed and saved: cleaned_images_1/cleaned_biassub_Light 4x4 8.000secs V NGC 2516 00025548.fit
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
dark sub biassub_Light 4x4 8.000secs B NGC 2516 00025554.fit 285.55902234546676
flat fielded biassub_Light 4x4 8.000secs B NGC 2516 00025554.fit 282.